In [1]:
import pandas as pd
import os 
os.getcwd()

'/Users/jfarland/Documents/research/solution-engineering/sales-engineering/megan/wave_apps/ts_clustering'

In [2]:
df = pd.read_csv("s3://h2o-public-test-data/bigdata/server/energy-data-science/energy-df-sample.tar.gz")

In [4]:
#df = df[['Store', 'Date', 'Weekly_Sales']].groupby(['Store', 'Date']).sum('Weekly_Sales').reset_index()
print(df.columns)
df.head()


Index(['energy-df-sample.csv', 'date', 'hour', 'LCLid', 'kw', 'visibility',
       'windBearing', 'temperature', 'timestamp', 'dewPoint', 'pressure',
       'apparentTemperature', 'windSpeed', 'precipType', 'icon', 'humidity',
       'summary', 'time'],
      dtype='object')


,energy-df-sample.csv,date,hour,LCLid,kw,visibility,windBearing,temperature,timestamp,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary,time
0,5.0,2011-11-24,0.0,MAC000150,0.386,8.88,217.0,9.01,2011-11-24 00:00:00,8.33,1029.52,7.37,2.92,rain,clear-night,0.96,Clear,00:00:00
1,18.0,2011-11-24,1.0,MAC000150,0.362,9.91,222.0,8.99,2011-11-24 01:00:00,8.04,1029.40,7.51,2.67,rain,clear-night,0.94,Clear,01:00:00
2,31.0,2011-11-24,2.0,MAC000150,0.314,9.27,215.0,8.56,2011-11-24 02:00:00,7.97,1029.38,7.01,2.65,rain,clear-night,0.96,Clear,02:00:00
3,44.0,2011-11-24,3.0,MAC000150,0.272,8.21,221.0,9.03,2011-11-24 03:00:00,8.36,1029.18,7.32,3.05,rain,clear-night,0.96,Clear,03:00:00
4,57.0,2011-11-24,4.0,MAC000150,0.160,8.93,215.0,8.78,2011-11-24 04:00:00,8.19,1028.89,7.01,3.06,rain,clear-night,0.96,Clear,04:00:00


In [5]:
df["sensor_id"] = df["LCLid"]
df = df[['timestamp', 'kw', 'sensor_id']].groupby(['timestamp', 'sensor_id']).sum('kw').reset_index()
df.head()

,timestamp,sensor_id,kw
0,2011-11-24 00:00:00,MAC000150,0.386
1,2011-11-24 01:00:00,MAC000150,0.362
2,2011-11-24 02:00:00,MAC000150,0.314
3,2011-11-24 03:00:00,MAC000150,0.272
4,2011-11-24 04:00:00,MAC000150,0.160


In [6]:
from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters

settings = MinimalFCParameters()

extracted_features = extract_features(df, column_id="sensor_id", column_sort="timestamp", default_fc_parameters=settings)

Feature Extraction: 100%|██████████| 25/25 [00:06<00:00,  4.05it/s]


In [7]:
extracted_features.head()

,kw__sum_values,kw__median,kw__mean,kw__length,kw__standard_deviation,kw__variance,kw__root_mean_square,kw__maximum,kw__absolute_maximum,kw__minimum
MAC000002,6023.221001,0.3410,0.504035,11950.0,0.445471,0.198444,0.672678,4.780,4.780,0.135
MAC000010,13738.266997,0.8730,1.100999,12478.0,0.777066,0.603831,1.347602,7.582,7.582,0.244
MAC000041,6736.777000,0.2250,0.351790,19150.0,0.350721,0.123005,0.496751,2.947,2.947,0.000
MAC000061,4701.709000,0.1705,0.243385,19318.0,0.179528,0.032230,0.302434,1.678,1.678,0.000
MAC000069,6402.874000,0.2660,0.330215,19390.0,0.257220,0.066162,0.418574,4.466,4.466,0.032


In [159]:
#df = df.set_index("sensor_id")
#df = df.pivot_table(index='sensor_id',columns='Date', values='Weekly_Sales', aggfunc='sum').reset_index()
#df = df.drop(columns=['Store'])

In [8]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "15.0.5" 2021-10-19; OpenJDK Runtime Environment Zulu15.36+13-CA (build 15.0.5+3-MTS); OpenJDK 64-Bit Server VM Zulu15.36+13-CA (build 15.0.5+3-MTS, mixed mode)
  Starting server from /Users/jfarland/opt/anaconda3/envs/ts_clustering/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/01/4mysj8cx1bjg_w8rbw097jwr0000gp/T/tmp28zya9gr
  JVM stdout: /var/folders/01/4mysj8cx1bjg_w8rbw097jwr0000gp/T/tmp28zya9gr/h2o_jfarland_started_from_python.out
  JVM stderr: /var/folders/01/4mysj8cx1bjg_w8rbw097jwr0000gp/T/tmp28zya9gr/h2o_jfarland_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.4
H2O_cluster_version_age:,1 month and 9 days
H2O_cluster_name:,H2O_from_python_jfarland_agvcpi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


In [9]:
df_h2o = h2o.H2OFrame(extracted_features)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [12]:
df_h2o.head()

kw__sum_values,kw__median,kw__mean,kw__length,kw__standard_deviation,kw__variance,kw__root_mean_square,kw__maximum,kw__absolute_maximum,kw__minimum
6023.22,0.341,0.504035,11950,0.445471,0.198444,0.672678,4.78,4.78,0.135
13738.3,0.873,1.101,12478,0.777066,0.603831,1.3476,7.582,7.582,0.244
6736.78,0.225,0.35179,19150,0.350721,0.123005,0.496751,2.947,2.947,0
4701.71,0.1705,0.243385,19318,0.179528,0.0322302,0.302434,1.678,1.678,0
6402.87,0.266,0.330215,19390,0.25722,0.0661619,0.418574,4.466,4.466,0.032
4952.03,0.189,0.31602,15670,0.283594,0.0804254,0.42461,3.324,3.324,0.049
9063.55,0.316,0.469176,19318,0.469113,0.220067,0.663471,4.26,4.26,0.073
2772.24,0.141,0.143863,19270,0.109523,0.0119953,0.180809,0.563,0.563,0.01
19639.2,0.8535,1.03725,18934,0.712833,0.508131,1.25858,5.717,5.717,0.172
8269.33,0.195,0.432904,19102,0.501288,0.25129,0.662341,4.782,4.782,0.062


In [10]:
from h2o.estimators.kmeans import H2OKMeansEstimator

kmeans_default = H2OKMeansEstimator(estimate_k = True, # Let Kmeans chose the optimal number of clusters
                                    k = 20, # Max number of clusters
                                    model_id = "kmeans_default.hex")

kmeans_default.train(training_frame = df_h2o, x=[i for i in df_h2o.col_names if i != 'sensor_id']);

kmeans Model Build progress: |███████████████████████████████████████████████████| (done) 100%


In [11]:
def get_table_data(data, model):
    clusters = model.predict(data)
    clusters.col_names = ['cluster']
    ext_data = clusters.cbind(data)
    
    return ext_data

In [12]:
def get_cluster_plot(data, model):
    ext_data = data[[i for i in df_h2o.col_names if i != 'sensor_id']].cbind(model.predict(data))
    
    ts_clusters = ext_data.group_by('predict').mean().get_frame()
    ts_clusters.col_names = [i.replace("mean_", "") for i in ts_clusters.col_names]
    ts_clusters.col_names = ["cluster"] + ts_clusters.col_names[1::]
    
    return ts_clusters

In [15]:
clusters = get_cluster_plot(df_h2o, kmeans_default)
clusters

kmeans prediction progress: |████████████████████████████████████████████████████| (done) 100%


cluster,kw__sum_values,kw__median,kw__mean,kw__length,kw__standard_deviation,kw__variance,kw__root_square,kw__maximum,kw__absolute_maximum,kw__minimum
0,2788.73,0.179144,0.261356,10857.6,0.261368,0.0778989,0.374107,2.80702,2.80702,0.0237797
1,6786.37,0.308615,0.437901,15597.1,0.384253,0.156589,0.587769,3.87211,3.87211,0.0612095
2,9903.27,0.377527,0.654617,15275.6,0.677893,0.477844,0.947185,5.83189,5.83189,0.0477091
3,12890.5,0.655867,0.826331,15573.3,0.581848,0.355955,1.0136,5.1849,5.1849,0.151467
4,16879.4,0.709875,1.12194,15084.3,1.14059,1.32089,1.60961,8.56683,8.56683,0.0676667
5,36931.8,2.2295,2.61298,14134,1.33108,1.77178,2.93248,8.971,8.971,0.25
6,3132.04,0.13982,0.196363,15884.3,0.182143,0.0383751,0.271871,2.15668,2.15668,0.0209892
7,18306.1,0.957333,1.27347,14566.4,0.875925,0.776978,1.54904,7.20956,7.20956,0.272556


In [107]:
h2o.export_file(clusters, "/Users/megankurka/AutoML/h2o3_apps/clustering/static/ts_clusters.csv", force=True)

Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


In [174]:
tbl_data = get_table_data(df_h2o, kmeans_default)
h2o.export_file(tbl_data, "/Users/megankurka/AutoML/h2o3_apps/clustering/static/data.csv", force=True)

kmeans prediction progress: |████████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


In [175]:
df = pd.read_csv("/Users/megankurka/AutoML/h2o3_apps/clustering/static/data.csv")

In [177]:
df = df[df.Store == 1]

In [181]:
df = df[[i for i in df.columns if i not in ['cluster', 'Store']]].transpose().reset_index()
df.columns = ['Date', 'Weekly_Sales']

,index,0
0,2010-02-05,1643690.90
1,2010-02-12,1641957.44
2,2010-02-19,1611968.17
3,2010-02-26,1409727.59
4,2010-03-05,1554806.68
...,...,...
130,2012-08-03,1631135.81
131,2012-08-10,1592409.96
132,2012-08-17,1597868.07
133,2012-08-24,1494122.29


In [139]:
plot_data = pd.read_csv("/Users/megankurka/AutoML/h2o3_apps/clustering/static/ts_clusters.csv")

In [140]:
plot_data = plot_data.transpose().reset_index()
plot_data = plot_data.iloc[1::]
plot_data.columns = ['Date'] + ["cluster: {}".format(i) for i in plot_data.columns[1::]]
plot_data = pd.melt(plot_data, id_vars=['Date'], value_vars=['cluster: 0', 'cluster: 1', 'cluster: 2', 'cluster: 3'])

In [141]:
plot_data.head()

,Date,variable,value
0,2010-02-05,cluster: 0,483979.996250
1,2010-02-12,cluster: 0,484765.011875
2,2010-02-19,cluster: 0,474784.978125
3,2010-02-26,cluster: 0,450464.778750
4,2010-03-05,cluster: 0,474192.820625


In [132]:
import plotly.express as px
fig = px.line(plot_data, x="Date", y=plot_data.columns,
              title='Clusters')
fig.update_yaxes(
    title='Weekly_Sales'
    )
fig.show()

In [113]:
df.head()

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
